In [1]:
import os
import os.path as osp

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
import pickle as pkl
import torchvision.models as models


In [18]:
im_path = 'data/middlebury/images/'
out_path = 'data/middlebury/features/'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
features_image1 = out_path + 'Backyard/frame07.pkl'
features_image2 = out_path + 'Backyard/frame08.pkl'
image1 = im_path + 'Backyard/frame07.png'
image2 = im_path + 'Backyard/frame08.png'

features1 = pkl.load( open(features_image1, "rb" ) )
features2 = pkl.load( open(features_image2, "rb" ) )

tensor_F1 = torch.from_numpy(features1["F"]).t()
tensor_U1 = torch.from_numpy(features1["U"]).t()
tensor_F2 = torch.from_numpy(features2["F"]).t()
tensor_U2 = torch.from_numpy(features2["U"]).t()


In [4]:
class DataLoader(data.Dataset):
	def __init__(self, data, args, pad_index, shuffle = True):
        super(DataLoader).__init__(self):
        classes, class_to_idx = self._find_classes(self.root)
        samples = self.make_dataset(self.root, class_to_idx, extensions, is_valid_file)
        
    def make_dataset( self,dir, class_to_idx, extensions=None, is_valid_file=None):
    images = []
    dir = os.path.expanduser(dir)
    for target in sorted(class_to_idx.keys()):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                if is_valid_file(path):
                    item = (path, class_to_idx[target])
                    images.append(item)

    return images

    def _find_classes(self, dir):
        """
        Finds the class folders in a dataset.
        Args:
            dir (string): Root directory path.
        Returns:
            tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
        Ensures:
            No class is a subdirectory of another.
        """
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

	def __getitem__(self):
        

	def __len__(self):
		return len(self.data)

torch.Size([100, 1536])


In [21]:
vgg_bn = models.vgg16_bn(pretrained = True)
class VGG_graph_matching(nn.Module):
    def __init__(self):
        super(VGG_graph_matching, self).__init__()
        self.features1 = nn.Sequential(
            *list(vgg_bn.features.children())[:33]
        )
        self.features2 = nn.Sequential(
            *list(vgg_bn.features.children())[33:43]
        )
                # => TODO: lambda init for trivial test
        self.lam = nn.Parameter(torch.ones(20, 2, device = device))
    def forward(self, x_1, mask_1, x_2 = None, mask_2 = None):
        
        x_1 = self.features1(x_1)
        U1 = x_1[:,:, mask_1[0]]
        x_1 = self.features2(x_1)
        F1 =  x_1[:,:, mask_1[1]]
        
        if x_2 is None:
            return U1, F1
        
        else:
            x_2 = self.features1(x_2)
            U2 = x_2[:,:, mask_2[0]]
            x_2 = self.features2(x_2)
            F2 =  x_2[:,:, mask_2[1]]
            
            test = torch.from_numpy(np.asarray([[1,0,1],[1,1,1],[1,0,0]]))
            [G, H] = buildGraphStructure(test)
            
            M = self.affinityMatrix_forward(F1, F2, U1, U2, G, G, H, H) #TODO: Build appropriate graph structure before using this
            v = self.powerIteration_forward(M)
            S = self.biStochastic_forward(v, G.shape[0], G.shape[0])
            d = self.voting_forward(S, P) #Have to get P still
            return d
    def kronecker(self, matrix1, matrix2):
        return torch.ger(matrix1.view(-1), matrix2.view(-1)).reshape(*(matrix1.size() + matrix2.size())).permute([0, 2, 1, 3]).reshape(matrix1.size(0) * matrix2.size(0), matrix1.size(1) * matrix2.size(1))


    def buildGraphStructure(self, A):
        """
        Arguments:
        ----------
            - A: node-to-node adjaceny matrix

        Returns:
        --------
            - G and H: node-edge incidence matrices such that: A = G*H^T

        """

        # Get number of nodes
        n = A.shape[0]

        # Count number of ones in the adj. matrix to get number of edges
        nr_edges = torch.sum(A)

        # Init G and H
        G = torch.zeros(n, nr_edges) 
        H = torch.zeros(n, nr_edges)

        # Get all non-zero entries and build G and H
        entries = (A != 0).nonzero()
        for count, (i,j) in enumerate(entries, start=0):
            G[i, count] = 1
            H[j, count] = 1

        return [G, H]


    def affinityMatrix_forward(self, F1, F2, U1, U2, G1, G2, H1, H2):
        """
        Arguments:
        ----------
            - F1, F2: edge features of input image 1 and 2
            - U1, U2: node features of input image 1 and 2
            - G1, H1: node-edge incidence matrices of image 1
            - G2, H2: node-edge incidence matrices of image 2

        Returns:
        ----------
            - M: affinity Matrix computed according to the given formula

        """



        # (a) Build X and Y
        #     - Get ordering of edges from G and H
        #     - Extract edge features of start and end nodes and concat
        idx1_start = (G1 != 0).nonzero()[:,0]
        idx2_start = (G2 != 0).nonzero()[:,0]
        idx1_end = (H1 != 0).nonzero()[:,0]
        idx2_end = (H2 != 0).nonzero()[:,0]
        X = torch.cat((F1[np.array(idx1_start)], F1[np.array(idx1_end)]), 1)
        Y = torch.cat((F2[np.array(idx2_start)], F2[np.array(idx2_end)]), 1)

        # (b) Calculate M_e = X * \lambda * Y^T
        M_e = torch.mm(torch.mm(X, self.lam), Y)

        # (c) Calculate M_p = U1 * U2^T
        M_p = torch.mm(U1, U2.t())

        # Calculate M = [vec(M_p)] + (G_2 \kronecker G_1)[vec(M_e)](H_2 \kronecker H_1)^T
        diagM_p = torchl.diag(M_p.view(M_p.numel()))
        diagM_e = torchl.diag(M_e.view(M_e.numel()))
        M = diagM_p + torch.mm(torch.mm(kronecker(G2, G1), diagM_e), kronecker(H2, H1).t())

        return M

    def powerIteration_forward(self, M, N = 1):
        """
        Arguments:
        ----------
            - M: affinity matrix

        Returns:
        --------
            - v*: optimal assignment vector (computed using power iterations) 
        """

        # Init starting v
        v = torch.ones(M.shape[1],1)

        # Perform N iterations: v_k+1 = M*v_k / (||M*v_k||_2) 
        for i in range(N):
            v = torch.mv(M, v)
            v = v / torch.norm(v, 2)

        return tensor_v    


    def biStochastic_forward(self, v, n, m, N = 1):
        """
        Arguments:
        ----------
            - v:     optimal assignment vector
            - n, m:  dimension of nodes of image 1 and image 2

        Returns:
        --------
            - S:    double stochastic confidence matrix S

        """

        # Reshape the assignment vector to matrix form
        S = v.view(n,m)

        # Perform N iterations: S_k+1 = ...., S_k+2 = ...
        for i in range(N):
            S = torch.mm(S, torch.mm(torch.ones(1,n),S).inverse())
            S = torch.mm(torch.mv(S, torch.ones(m,1)).inverse(), S)

        return S

    def voting_forward(self, S, P, alpha = 200., th = 10):
        """
        Arguments:
        ----------
            S - confidence map obtained form bi-stochastic layer
            P - Position matrix (m x 2)
            alpha - scaling factor
            th - number of pixels to be set as threshold beyond which confidence levels are set to zero.
        Returns:
        --------
            - d: displacement vector

        """
        S_ = alpha*S
        #TODO: Apply th
        P_ = torch.mm(F.softmax(S, dim = -1), P) 
        d = torch.zeros(P.shape)
        for i in range(P.shape[0]):
            d[:, i] = P_ -  P[:, i]

        return d

def loss(d, d_gt):
    return torch.sum(torch.sqrt(torch.inner(d - d_gt, (d-d_gt).t()) + 1e-6))

In [6]:
# Some tests
# -----------
test = torch.from_numpy(np.asarray([[1,0,1],[1,1,1],[1,0,0]]))
[G, H] = buildGraphStructure(test)
print(G)
print(H)

tensor([[1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])
tensor([[1., 0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0.]])


In [22]:


model = VGG_graph_matching().to(device)
transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    ])

lam
features1.0.weight
features1.0.bias
features1.1.weight
features1.1.bias
features1.3.weight
features1.3.bias
features1.4.weight
features1.4.bias
features1.7.weight
features1.7.bias
features1.8.weight
features1.8.bias
features1.10.weight
features1.10.bias
features1.11.weight
features1.11.bias
features1.14.weight
features1.14.bias
features1.15.weight
features1.15.bias
features1.17.weight
features1.17.bias
features1.18.weight
features1.18.bias
features1.20.weight
features1.20.bias
features1.21.weight
features1.21.bias
features1.24.weight
features1.24.bias
features1.25.weight
features1.25.bias
features1.27.weight
features1.27.bias
features1.28.weight
features1.28.bias
features1.30.weight
features1.30.bias
features1.31.weight
features1.31.bias
features2.1.weight
features2.1.bias
features2.2.weight
features2.2.bias
features2.4.weight
features2.4.bias
features2.5.weight
features2.5.bias
features2.7.weight
features2.7.bias
features2.8.weight
features2.8.bias


In [73]:
def get_mask(width, height, grid_size = 10): 
    """
    Get the location based on the image size corresponding to relu_4_2 
    and relu_5_1 layer for a desired grid size. 
    """
    x_jump = int(width/grid_size) 
    y_jump = int(height/grid_size)
    x_idx = np.linspace(int(x_jump/2),int(width - x_jump/2), grid_size, dtype = np.int32)
    y_idx = np.linspace(int(y_jump/2), int(height - y_jump/2), grid_size, dtype = np.int32)
    f_mask = torch.zeros((height//(2**4),width//2**4)).byte()
    u_mask = torch.zeros((height//(2**3),width//2**3)).byte()
    for i in x_idx:
        for j in y_idx:
            f_mask[j//(2**4),i//(2**4)] = 1
            u_mask[j//(2**3),i//(2**3)] = 1
    return(u_mask, f_mask)

In [74]:
optim_vgg = nn.optim.Adam(model.parameters(), lr = 1e-3)

im1 = Image.open(image1).convert('RGB') 
im2 = Image.open(image2).convert('RGB') 
im1 = transformation(im1).unsqueeze_(0)
im2 = transformation(im2).unsqueeze_(0)
mask_1 = get_mask(im1.size[0],im1.size[1])
mask_1 = get_mask(im2.size[0],im2.size[1])

d = model(im1, mask1, im2, mask2) 

loss = loss(d, d_gt) #To be implemented
loss.backward()
optim_vgg.step()